In [1]:
from src.data import extract_vault, holdout, delete_nodes_without_links
import matplotlib.pyplot as plt 
import networkx as nx

import markdown as md
from bs4 import BeautifulSoup
import PyPDF2

import io
from PIL import Image
import numpy as np

import torch
from transformers import BertTokenizer, BertModel, VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

from tqdm.notebook import tqdm
import pickle

from scipy import sparse
import pandas as pd

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# vault = extract_vault('./vaultes/obsidian-hub/')
vault = extract_vault('./vaultes/brain/')

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, GitHub wikis in obsidian & interwikilinks to placeholder-1671408403833.jpeg
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   rel_filepath      124 non-null    object        
 1   abs_filepath      124 non-null    object        
 2   file_exists       135 non-null    object        
 3   n_backlinks       135 non-null    object        
 4   n_wikilinks       114 non-null    float64       
 5   n_tags            114 non-null    float64       
 6   n_embedded_files  114 non-null    float64       
 7   modified_time     124 non-null    datetime64[ns]
 8   graph_category    135 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 10.5+ KB
None


In [3]:
#df = brain_vault.get_all_file_metadata()
df = vault.get_all_file_metadata()
df

,rel_filepath,abs_filepath,file_exists,n_backlinks,n_wikilinks,n_tags,n_embedded_files,modified_time,graph_category
file,,,,,,,,,
GitHub wikis in obsidian & interwikilinks,NaN,NaN,False,1,NaN,NaN,NaN,NaT,nonexistent
MultiMarkdown,varia/MultiMarkdown.md,vaultes/brain/varia/MultiMarkdown.md,True,0,0.0,1.0,0.0,2023-03-23 17:03:45.743535995,note
bring value ASAP,tech art/learn/bring value ASAP.md,vaultes/brain/tech art/learn/bring value ASAP.md,True,1,0.0,1.0,0.0,2023-03-23 17:03:45.741283655,note
blueprint python,unreal/blueprint python.md,vaultes/brain/unreal/blueprint python.md,True,0,0.0,1.0,0.0,2023-03-23 17:03:45.743141890,note
install apps winget,varia/install apps winget.md,vaultes/brain/varia/install apps winget.md,True,0,0.0,1.0,0.0,2023-03-23 17:03:45.744144678,note
...,...,...,...,...,...,...,...,...,...
Obsidian private comments-1673286147963.jpeg,image/Obsidian private comments-1673286147963....,vaultes/brain/image/Obsidian private comments-...,True,1,NaN,NaN,NaN,2023-03-23 17:03:45.735747337,attachment
add notes to explorer-1673900509482.jpeg,image/add notes to explorer-1673900509482.jpeg,vaultes/brain/image/add notes to explorer-1673...,True,1,NaN,NaN,NaN,2023-03-23 17:03:45.735861540,attachment
navigate sibling page-1674170180221.jpeg,image/navigate sibling page-1674170180221.jpeg,vaultes/brain/image/navigate sibling page-1674...,True,1,NaN,NaN,NaN,2023-03-23 17:03:45.736437798,attachment


In [4]:
df.index.value_counts()

GitHub wikis in obsidian & interwikilinks    1
file URI                                     1
subsequent-sibling combinator                1
TA background                                1
learn shortcuts fast                         1
                                            ..
minimal notetaking                           1
run MaxScript                                1
max dark script editor listener              1
note-link-janitor                            1
placeholder-1671408403833.jpeg               1
Name: file, Length: 135, dtype: int64

In [5]:
len(vault.graph.nodes)

135

In [6]:
delete_nodes_without_links(vault.graph)

deleted 39 nodes from graph


In [7]:
len(vault.graph.nodes)

96

In [8]:
def get_text_from_html(html):
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

def get_text_from_markdown(filepath):
    with open(filepath, 'r') as f:
        text = f.read()
        html = md.markdown(text)
        text = get_text_from_html(html)
    return text

def get_text_from_pdf(filepath):
    with open(filepath, 'rb') as f:
        pdfReader = PyPDF2.PdfReader(f)
        text = ""
        
        for page in pdfReader.pages:
            text += page.extract_text()
            all_images.extend(page.images)
    return text


def get_images_from_pdf(filepath):
    with open(filepath, 'rb') as f:
        pdfReader = PyPDF2.PdfReader(f)
        images = []
        
        for page in pdfReader.pages:
            for image_file in page.images:
                images.append(Image.open(io.BytesIO(image_file.data)))
    return images

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer_bert = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_bert = BertModel.from_pretrained("bert-base-multilingual-cased")

model_bert.to(device)

1

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1

In [10]:
# model_i2t = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# tokenizer_i2t = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# model_i2t.to(device)

# 1

In [11]:
@torch.no_grad()
def encode_text(text):
    encoded_input = tokenizer_bert(text, truncation=True, return_tensors='pt')
    output = model_bert(**encoded_input)
    return output.pooler_output.cpu().numpy()[0]

In [12]:
# def image_to_text(image):
#     if image.mode != "RGB":
#         image = image.convert(mode="RGB")
    
#     pixel_values = feature_extractor(images=[image], return_tensors="pt").pixel_values
#     pixel_values = pixel_values.to(device)

#     gen_kwargs = {"max_length": 256, "num_beams": 4}
#     output_ids = model_i2t.generate(pixel_values, **gen_kwargs)

#     preds = tokenizer_i2t.batch_decode(output_ids, skip_special_tokens=True)
#     preds = [pred.strip() for pred in preds]
#     return preds[0]

In [13]:
# embeddings = []

# for path in tqdm(df["abs_filepath"]):
#     extension = str(path).split('.')[-1]
#     if extension == 'md':
#         text = get_text_from_markdown(path)
#         embeddings.append(encode_text(text))
#     elif extension == 'pdf':
#         text = get_text_from_pdf(path)
#         images = get_images_from_pdf(path)
#         for image in tqdm(images, desc='images', leave=False):
#             text += " "
#             text += image_to_text(image)
#         embeddings.append(encode_text(text))
#     elif extension in {'jpeg', 'gif', 'png'}:
#         image = Image.open(path)
#         text = image_to_text(image)
#         embeddings.append(encode_text(text))
#     else:
#         # embeddings.append(np.random.rand(768))
#         1/0

In [14]:
embeddings = []

for text in tqdm(vault.graph.nodes):
    embeddings.append(encode_text(text))

  0%|          | 0/96 [00:00<?, ?it/s]

In [15]:
emb_dict = {}

for file, emb in zip(vault.graph.nodes, embeddings):
    emb_dict[file] = emb

with open("brain_embeddings.pkl", 'wb') as f:
    pickle.dump(emb_dict, f)

# LightFM part

In [12]:
from lfm import build_lfm_model
from evaluation import topn_recommendations

/Users/avzelentsov/PycharmProjects/LinkPredict/.rec_sys_proj_venv/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [13]:
len(vault.graph.nodes)

96

In [14]:
modified_graph, deleted_edges = holdout(vault.graph, alpha=0.1)

In [15]:
modified_graph

In [16]:
del_edges = []

for edge in deleted_edges:
    ind0 = list(vault.graph.nodes).index(edge[0])
    ind1 = list(vault.graph.nodes).index(edge[1])
    del_edges.append((ind0, ind1))

del_edges

[(73, 2),
 (63, 2),
 (61, 18),
 (59, 44),
 (47, 58),
 (42, 10),
 (40, 4),
 (29, 72),
 (26, 64),
 (26, 81),
 (14, 25),
 (12, 51),
 (11, 15)]

In [17]:
train_matrix = nx.adjacency_matrix(modified_graph)
train_matrix = train_matrix
train_matrix.shape

(96, 96)

In [18]:
train_matrix.sum()

117

In [19]:
data_description = dict(
    n_users = train_matrix.shape[0],
    n_items = train_matrix.shape[1],
    # user_features = ... skip for now, will be defined later
    # item_features = None # compound features are not present in the dataset
)
data_description

{'n_users': 96, 'n_items': 96}

In [20]:
with open("brain_embeddings.pkl", 'rb') as f:
    emb_dict = pickle.load(f)

In [21]:
data_description["user_features"] = sparse.csr_matrix([emb_dict[file] for file in vault.graph.nodes])
data_description["item_features"] = sparse.csr_matrix([emb_dict[file] for file in vault.graph.nodes])

In [22]:
holdout_data_description = dict(
    users = [de_[0] for de_ in del_edges],
    n_items = train_matrix.shape[1],
    # user_features = ... skip for now, will be defined later
    # item_features = None # compound features are not present in the dataset
)

In [23]:
holdout_data_description["user_features"] = sparse.csr_matrix([emb_dict[file] for file in vault.graph.nodes])
holdout_data_description["item_features"] = sparse.csr_matrix([emb_dict[file] for file in vault.graph.nodes])

In [24]:
lfm_config = dict(
    no_components = 256,
    loss = 'warp',
    max_sampled = 1,
    max_epochs = 30,
    learning_schedule = 'adagrad',
    user_alpha = 1e-3,
    item_alpha = 1e-3,
    random_state = 7032023
)

In [25]:
topn = 10

In [26]:
lfm_model = build_lfm_model(
    lfm_config,
    train_matrix,
    data_description,
    iterator = tqdm
)

  0%|          | 0/30 [00:00<?, ?it/s]

In [27]:
# your code here
def cartesian_product(x, y):
    return np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])

def lightfm_scoring(model, data, data_description, item_bias=True):
    users = np.unique(data_description['users'])
    items = np.arange(data_description['n_items'])
    
    user_item_pairs = cartesian_product(users, items)
    
    if item_bias is False:
        cur_biases = lfm_model.item_biases.copy()

        lfm_model.item_biases *= 0
    
    predictions = model.predict(
                                user_ids=user_item_pairs[:, 0],
                                item_ids=user_item_pairs[:, 1],
                                user_features=data_description['user_features'],
                                item_features=data_description['item_features'],
                               )
    scores = predictions.reshape(len(items), len(users))
    
    if item_bias is False:
        lfm_model.item_biases[:] = cur_biases
    
    return scores.T, users

In [28]:
lfm_scores, test_users = lightfm_scoring(lfm_model, None, holdout_data_description, item_bias=False)
lfm_recs = topn_recommendations(lfm_scores, topn=topn)

In [29]:
def recall(lfm_recs, users, del_edges, holdout_data_description):
    
    edges_dict = {}
    
    for edge in del_edges:
        edges_dict[edge[0]] = edges_dict.get(edge[0], set())
        edges_dict[edge[0]].add(edge[1])
    
    metric = 0
    
    for i, rec in enumerate(lfm_recs):
        metric += len(set(rec) & edges_dict[users[i]]) / len(edges_dict[users[i]])
    
    return metric / len(users)

In [30]:
recall(lfm_recs, test_users, del_edges, holdout_data_description)

0.3333333333333333

In [31]:
recalls = []

alpha = 0.1
topn = 5


for _ in tqdm(range(50)):
    modified_graph, deleted_edges = holdout(vault.graph, alpha=alpha)
    
    train_matrix = nx.adjacency_matrix(modified_graph)
    data_description = dict(
        n_users = train_matrix.shape[0],
        n_items = train_matrix.shape[1],
        # user_features = ... skip for now, will be defined later
        # item_features = None # compound features are not present in the dataset
    )
    
    with open("brain_embeddings.pkl", 'rb') as f:
        emb_dict = pickle.load(f)
    
    data_description["user_features"] = sparse.csr_matrix([emb_dict[file] for file in vault.graph.nodes])
    data_description["item_features"] = sparse.csr_matrix([emb_dict[file] for file in vault.graph.nodes])
    
    holdout_data_description = dict(
        users = [de_[0] for de_ in del_edges],
        n_items = train_matrix.shape[1],
        # user_features = ... skip for now, will be defined later
        # item_features = None # compound features are not present in the dataset
    )
    
    holdout_data_description["user_features"] = sparse.csr_matrix([emb_dict[file] for file in vault.graph.nodes])
    holdout_data_description["item_features"] = sparse.csr_matrix([emb_dict[file] for file in vault.graph.nodes])
    
    lfm_config = dict(
        no_components = 256,
        loss = 'warp',
        max_sampled = 1,
        max_epochs = 30,
        learning_schedule = 'adagrad',
        user_alpha = 1e-3,
        item_alpha = 1e-3,
        random_state = 7032023
    )
    
    lfm_model = build_lfm_model(
        lfm_config,
        train_matrix,
        data_description,
    )
    
    lfm_scores, test_users = lightfm_scoring(lfm_model, None, holdout_data_description, item_bias=False)
    lfm_recs = topn_recommendations(lfm_scores, topn=topn)
    
    recalls.append(recall(lfm_recs, test_users, del_edges, holdout_data_description))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [34]:
print(np.mean(recalls))

In [33]:
fig, ax = plt.subplots(1)
hist = pd.DataFrame(recalls).hist(bins=15, ax=ax)
plt.savefig(f'./results/lightfm/alpha={alpha}_topn={topn}_n_exp=50_vault=brain.png')